Temporary Workaround for 

In [ ]:
!pip install -q snscrape
!pip install -q vaderSentiment
!pip install -q transformers

     |████████████████████████████████| 125 kB 5.2 MB/s 
     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 67.6 MB/s 
     |████████████████████████████████| 7.6 MB 43.7 MB/s 


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import tweepy
from datetime import date
import textblob
import transformers

In [ ]:
today = date.today()
end_date = today
print(end_date)
search_term = 'IlliniMBB'
from_date = '2022-01-01'

2022-11-29


In [ ]:
os.system(f"snscrape --since {from_date} twitter-search '{search_term} until:{end_date}' > result-tweets.txt")
if os.stat("result-tweets.txt").st_size == 0:
  counter = 0
else:
  df = pd.read_csv('result-tweets.txt', names=['link'])
  counter = df.size

print('Number Of Tweets : '+ str(counter))

Number Of Tweets : 45406


In [ ]:
max_results = 100
extracted_tweets = "snscrape --format '{content!r}'"+ f" --max-results {max_results} --since {from_date} twitter-search '{search_term} until:{end_date}' > extracted-tweets.txt"
os.system(extracted_tweets)
if os.stat("extracted-tweets.txt").st_size == 0:
  print('No Tweets found')
else:
  df = pd.read_csv('extracted-tweets.txt', names=['content'])
  for row in df['content'].iteritems():
    print(row)

(0, "One year ago this week - Nov. 29, 2021 - Brad Underwood's @IlliniMBB team juggled a flurry of injuries and illness, but got 28 points from @kxng_alpha to defeat visiting Notre Dame, 82-72. https://t.co/sYzmjZbEoD")
(1, "'@JonSportsc @deebrown11 @DeronWilliams @Head2Lu @IlliniMBB @ArmchairIllini Great team. Only to lose to my heels 😏'")
(2, "'@AtomSer7 @IlliniMBB @MelendezRamses They started the season at #5.'")
(3, "We're excited to announce @TexasMBB will  play in the #JimmyVClassic on Dec. 6 supporting our game-changing cancer research at @DellMedSchool. \\n\\nTune in to @espn at 7 ET to watch the Longhorns battle @IlliniMBB at @TheGarden\\n\\nLearn more https://t.co/XRWlyvlx1I #VictoryOverCancer https://t.co/UjGRU8mAXZ")
(4, "'These 3 were special. @deebrown11 @DeronWilliams @Head2Lu @IlliniMBB @ArmchairIllini https://t.co/6JbRUGpOTl'")
(5, "'@Sam_Layendecker @skyyclark @Sam_Layendecker The Hail to the Orange Out is January 13th vs Michigan State!'")
(6, "'@rossy_burner @UMvsEv

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = df
list = df['content'].astype(str).tolist()


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
score_set = sentiment_pipeline(list)
average = 0
count = 0

for x in score_set:
  if (x['label'] == 'NEGATIVE'):
    average = average - (x['score'])
  else:
    average = average + (x['score'])
  count += 1

average = average / count
print("Average sentiment: " + str(average))

Average sentiment: -0.4420957750082016


API Tweepy

In [ ]:
api_key =  "qoHykDTyFDIN0GdlnUHsI5dek"
api_key_secret = "tt3Xe8Za0yuQTjqiXoVWWcmLqfejh41SljZf0uVlvE8JxdPzo5"
access_token = "1587238601766719489-b6AZWjTRlpDi2gFe0spDBeXm8DYt3L"
access_token_secret =   "89MwP7atYbn34psj2VRQh9sCfekLxLSl7pJjUHMbEBbwM"

In [ ]:
authenticator = tweepy.OAuthHandler(api_key, api_key_secret)
authenticator.set_access_token(access_token, access_token_secret)

In [ ]:
api = tweepy.API(authenticator, wait_on_rate_limit=True)

# posts = api.user_timeline(screen_name="BillGates", count = 100, lang="en", tweet_mode="extended")
# print("Show 5 most recent tweets on timeline: /n")
# for tweet in posts[0:5]:
#   print(tweet.full_text + '/n')

In [ ]:
team_dict = {
    "U Illinois": "@IlliniMBB",
    "U Indiana": "@IndianaMBB",
    "U Iowa": "@IowaHoops",
    "U Maryland": "@TerrapinHoops",
    "U Michigan": "@umichbball",
    "U Minnesota": "@GopherMBB",
    "U Nebraska": "@HuskerHoops",
    "U Wisconsin": "@BadgerMBB",
    "Michigan State": "@MSU_Basketball",
    "Penn State": "@PennStateMBB",
    "Ohio State": "@OhioStateHoops",
    "Rutgers": "@RutgersMBB",
    "Purdue": "@BoilerBall",
    "Northwestern": "@NUMensBball",
}

In [ ]:
team = 'Illini'

In [ ]:
search = f'#{team} -filter:retweets'
tweet_cursor = tweepy.Cursor(api.search, q=search, lang='en', tweet_mode='extended').items(500)
tweets = [tweet.full_text for tweet in tweet_cursor]
tweets_df = pd.DataFrame(tweets, columns=['Tweets'])

In [ ]:
for _, row in tweets_df.iterrows():
  row['Tweets'] = re.sub('http\S+', '', row['Tweets'])
  row['Tweets'] = re.sub('#\S+', '', row['Tweets'])
  row['Tweets'] = re.sub('@\S+', '', row['Tweets'])
  row['Tweets'] = re.sub('\\n', '', row['Tweets'])

In [ ]:
tweets_df = df
tweets_df['Polarity'] = tweets_df['Tweets'].map(lambda tweet: textblob.TextBlob(tweet).sentiment.Polarity)
tweets_df['Result'] = tweets_df['Polarity'].map(lambda pol: '+' if pol > 0 else '-')

In [ ]:
positive = tweets_df[tweets_df.Result == '+'].count()['Tweets']
positive = tweets_df[tweets_df.Result == '-'].count()['Tweets']

In [ ]:
plt.bar([0,1], [positive, negative], label=['Positive', 'Negative'], color = ['orange', 'blue'])
plt.show()